<a href="https://colab.research.google.com/github/kashyapsanket/Age-Detection-From-Blogs/blob/master/Feature_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import re
import nltk
from collections import Counter
import math

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
POS_dic = {"CC":1, "CD":2, "DT":3, "EX":4, "FW":5, "IN":6, "JJ":7, "JJR":8, "JJS":9, "LS":10, "MD":11, "NN":12, "NNS":13, "NNP":14, "NNPS":15, "PDT":16, "POS":17,\
"PRP":18, "PRP$":19, "RB":20, "RBR":21, "RBS":22, "RP":23, "SYM":24, "TO":25, "UH":26, "VB":27, "VBD":28, "VBG":29, "VBN":30, "VBP":31, "VBZ":32, "WDT":33, "WP":34, "WP$":35, "WRB":36}

In [0]:
def get_sentence_information(text):
    #the list of feature to be returned
    # #4 features
    text = re.sub("!{2,}","!", text)
    text = re.sub("\.{2,}",".", text)
    text = re.sub("\?{2,}","?", text)

    tokens = nltk.word_tokenize(text) #tokenise the string
    to_return = []

    split = ['.','?', '!']
    lower_count = 0
    upper_count = 0
    current_sentence_count = 0
    sentence_count = 0
    word_count = 0

    punc =['.',',','?','!',';',':']
    for token in tokens:
        if token in split:
            if current_sentence_count > 0:
                sentence_count += 1
            current_sentence_count = 0
        elif token not in punc:
            current_sentence_count+=1
            if current_sentence_count == 1:
                if token[0].isupper():
                    upper_count+=1
                else:
                    lower_count+=1
                word_count+=1

    if sentence_count == 0:
        sentence_count = 1
    to_return.append(sentence_count) #total no of sentences
    to_return.append(float(word_count)/sentence_count) #average word count of sentences
    to_return.append(float(upper_count)/sentence_count) # % of sentences with upper case starting
    to_return.append(float(lower_count)/sentence_count) # % of sentences with lower case starting
    return to_return

In [0]:
def get_lines_information(data_string):
    #the list of features to be returned
    # #4 features
    #print("Line wise")
    to_return = []
    to_return.append(data_string.count("\n")) #count the number of line
    line_data = data_string.split("\n")
    punc =['.',',','?','!',';',':']
    char_count = 0
    word_count = 0
    empty_count = 0
    for i in range(len(line_data)):
        if line_data[i] == "":
            empty_count+=1
        else:
            char_count+=len(line_data[i])
            tokens = nltk.word_tokenize(line_data[i])
            for j in tokens:
                if j not in punc:
                    word_count+=1
    to_return.append(float(empty_count)/to_return[0]) #count the % of empty lines
    to_return.append(float(char_count)/to_return[0]) #count the average no of characters in a line
    to_return.append(float(word_count)/to_return[0]) #count the average no fo words in a line
    return to_return

In [0]:
def character_features(data_string):
    #the list to be returned
    # #7 features
    to_return = []
    to_return.append(len(data_string)) #total no of characters
    to_return.append(float(sum(c.isalpha() and c.islower() for c in data_string))/to_return[0]) # total no of letters
    to_return.append(float(sum(c.isdigit() for c in data_string))/to_return[0]) # total no of digital characters
    to_return.append(float(sum(c.isupper() for c in data_string))/to_return[0]) #total no of upper case characters
    to_return.append(float(sum(c.isspace() for c in data_string))/to_return[0]) #total no of whitespace characters
    to_return.append(float(data_string.count('\t'))/to_return[0]) # total no of tab space characters
    to_return.append(float(1-to_return[1]-to_return[2]-to_return[3]-to_return[4]-to_return[5])) # total no of special characters
    return to_return

In [0]:
def function_words_features(data_string):
    # total 7 features
    tokened_data_string = nltk.word_tokenize(data_string)
    total_words = len(tokened_data_string)
    if total_words == 0:
        return [0, 0, 0, 0, 0, 0, 0]
    to_return = []
    postagtext = nltk.pos_tag(tokened_data_string)
    to_return.append(float(str(postagtext).count('DT'))/float(total_words))
    to_return.append(float((data_string.lower().count('yes') + data_string.lower().count('no') + data_string.lower().count('okay') + data_string.lower().count('ok')))/float(total_words))
    total_pronouns = str(postagtext).count('PP') + str(postagtext).count('PP$') + str(postagtext).count('WP') + str(postagtext).count('WP$')
    to_return.append(float(total_pronouns)/float((total_words)))
    to_return.append(float((str(postagtext).count('VB') + str(postagtext).count('VBD') + str(postagtext).count('VBG') + str(postagtext).count('VBN') + str(postagtext).count('VBP') + str(postagtext).count('VBZ'))) / float(total_words))
    to_return.append(float(str(postagtext).count('CC'))/float(total_words))
    to_return.append(float(str(postagtext).count('UH'))/float(total_words))
    to_return.append(float(str(postagtext).count('IN'))/float(total_words))
    return to_return

In [0]:

def calculate_simpson_d_measure(data_word_count, no_of_words):
    to_return = 0
    for word_count in data_word_count.keys():
        if no_of_words == 1:
            continue
        to_return = to_return + float(len(data_word_count[word_count])*float(word_count)*float(word_count-1)/float(no_of_words*(no_of_words-1)))
    return to_return

def calculate_sichel_s_measure(di_hapaxs,word_set_len):
    to_return = 0
    if word_set_len == 0:
        return 0
    to_return = to_return + float(float(len(di_hapaxs))/word_set_len)
    return to_return

def calculate_honore_r_measure(hapaxs,word_set_len,no_of_words):
    to_return = 0
    to_return = to_return + float(100*math.log10(no_of_words))
    if word_set_len == 0:
        return to_return
    to_return = to_return/float(1-float(len(hapaxs))/word_set_len)
    return to_return

def calculate_entropy_measure(data_word_count,no_of_words):
    to_return = 0
    for word_count in data_word_count.keys():
        to_return = to_return + float(len(data_word_count[word_count])*math.log10(float(no_of_words)/word_count)*float(word_count)/no_of_words)
    return to_return
  
def calculate_yule_k(data_word_count,no_of_words):
    to_return = 0
    to_return = to_return + 10000.0/float(no_of_words)
    for word_count in data_word_count.keys():
        to_return = to_return + float(10000.0*(float(len(data_word_count[word_count]))*(float(word_count/no_of_words))**2))
    return to_return


In [0]:
def word_based_features(data_string):
    #the list of features to be returned
    # #currently 32 features
    punctuations = ['.',',','?','!',';',':']
    to_return = []
    word_len_count = [0]*20
    data_tokens = nltk.word_tokenize(data_string.lower()) #tokenise the string
    data_tokens = [i for i in data_tokens if i not in punctuations and i != '']
    if len(data_tokens) == 0:
        return [0]*31
    data_tokens_set = set(data_tokens)
    data_tokens_len = [float(len(i)) + 1 for i in data_tokens] #get tokens length
    data_tokens_larger_6 = [i for i in data_tokens if len(i)>6] #get tokens of length > 6
    data_tokens_small = [i for i in data_tokens if len(i) < 4] #get tokens of length < 4
    data_count = Counter(data_tokens)
    data_count_dict = dict(data_count) #get count of each token
    data_word_count = {}
    for i in data_count_dict.keys():
        try:
            data_word_count[data_count_dict[i]].append(i)
        except KeyError:
            data_word_count[data_count_dict[i]] = []
            data_word_count[data_count_dict[i]].append(i)
    hapaxs = [i for i in data_count_dict.keys() if data_count_dict[i] == 1] 
    di_hapaxs = [i for i in data_count_dict.keys() if data_count_dict[i] == 2]
    to_return.append(len(data_tokens)) #no of words
    to_return.append(sum(data_tokens_len)/float(len(data_tokens_len))) #average length of words
    to_return.append(float(len(set(data_tokens)))/float(len(data_tokens))) #no of distinct words, vocabulary strength
    to_return.append(float(len(data_tokens_larger_6))/float(len(data_tokens))) #no of words with length > 6
    to_return.append(float(len(data_tokens_small))/float(len(data_tokens))) #no of words with length < 4
    to_return.append(float(len(hapaxs))/float(len(data_tokens))) #hapax legomena
    to_return.append(float(len(di_hapaxs))/float(len(data_tokens))) #hapax dislegomena
    to_return.append(calculate_yule_k(data_word_count,len(data_tokens))) #yule's k measure
    to_return.append(calculate_simpson_d_measure(data_word_count,len(data_tokens))) #simpsons d measure
    to_return.append(calculate_sichel_s_measure(di_hapaxs,len(data_tokens_set))) #sichels s measures
    to_return.append(calculate_entropy_measure(data_word_count,len(data_tokens))) #calculate the entropy measure
    for i in data_tokens:
        try:
            word_len_count[len(i)]+=1
        except:
            pass
    to_return.extend(word_len_count) #20 features
    return to_return

In [0]:
def pos_tag_start(data_string):
    global POS_dic
    to_return = [0]*36
    tokened_data_string = nltk.word_tokenize(data_string)
    postagtext = nltk.pos_tag(tokened_data_string)
    punc =['.','?',';']
    sen = []
    sen_list1=[]
    for i in postagtext:
        if not i[0] in punc:
            sen.append(i)
        else:
            sen_list1.append(sen)
            sen=[]
    while [] in sen_list1:
        sen_list1.remove([])
    for i in sen_list1:
        try:
            to_return[POS_dic[i[0][1]]-1]+=1
        except KeyError:
            pass
    return to_return

In [0]:
 nltk.download('punkt')
 nltk.download('averaged_perceptron_tagger')
 df = pd.read_csv('gdrive/My Drive/BlogData.csv')
 text_df = df[['Text', 'Age']]
 #print(text_df)
 temp_df = text_df[0:100000]
 feature_set = []

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [0]:
for index, row in temp_df.iterrows():
  features = []
  features.extend(pos_tag_start(row['Text']))
  features.extend(get_sentence_information(row['Text']))
  features.extend(get_lines_information(row['Text']))
  features.extend(word_based_features(row['Text']))
  features.extend(function_words_features(row['Text']))
  features.extend(character_features(row['Text']))
  feature_set.append(features)
  if index%10000 == 0:
    print(index)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


In [0]:
feature_df = pd.DataFrame(feature_set)
feature_df.to_csv('normalizedfeatures.csv')

0       0
1       1
2       0
3       0
4       0
5       0
6       0
7       6
8       0
9       1
10      0
11      1
12      0
13      0
14      1
15      7
16      3
17      0
18      2
19      0
20      4
21      7
22      3
23      4
24      0
25      1
26      6
27      3
28      2
29      7
       ..
9969    0
9970    0
9971    2
9972    0
9973    0
9974    0
9975    0
9976    0
9977    0
9978    0
9979    0
9980    0
9981    0
9982    0
9983    0
9984    0
9985    0
9986    4
9987    4
9988    1
9989    5
9990    0
9991    0
9992    0
9993    1
9994    0
9995    1
9996    1
9997    0
9998    1
Name: 0, Length: 9999, dtype: int64